In [1]:
#importing gpu
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import tensorflow 
config = ConfigProto()

#checking gpu function availability
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True

#session-configuration adjusting
session = InteractiveSession(config=config)

#checking tensorflow-gpu support
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

#print(len(tensorflow.config.list_physical_devices('GPU'))>0)
#tf.debugging.set_log_device_placement(True)


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3865068929699059714
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4294737920
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16193610778694967488
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]
2.10.1
Num GPUs Available:  1


In [2]:
#checking gpu status
#tf.debugging.set_log_device_placement(False)
!nvidia-smi

Tue Dec 27 13:11:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.56       Driver Version: 527.56       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| 37%   59C    P2   140W / 250W |   7930MiB /  8192MiB |     62%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
#importing libraries
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
import scipy
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf

In [4]:
#calling global varibles

IMAGE_SIZE = [299, 299]
TRAIN_PATH = 'F:/FAHIM/Q4S10/fahim datasets/Bean/train'
VALIDATION_PATH = 'F:/FAHIM/Q4S10/fahim datasets/Bean/validation'

In [5]:
#Pretrained model structure fetching with custom dataset
base_model = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
base_model.trainable = True
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

Number of layers in the base model:  311


In [6]:
#Importing training dataset
folders = glob('F:/FAHIM/Q4S10/fahim datasets/Bean/train/*')

In [7]:
#Importing flattened structure as output layer
flatten = Flatten()(base_model.output)

In [8]:
#Calling layers
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(len(folders), activation="softmax")(bboxHead)

In [9]:
#Fixing input, output layers args
model = Model(inputs=base_model.input, outputs=bboxHead)

In [10]:
#Showing model status
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

 batch_normalization_5 (BatchNo  (None, 35, 35, 64)  192         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 35, 35, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_10 (BatchN  (None, 35, 35, 96)  288         ['conv2d_10[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_11 (BatchN  (None, 35, 35, 32)  96          ['conv2d_11[0][0]']              
 ormalizat

                                                                                                  
 batch_normalization_22 (BatchN  (None, 35, 35, 64)  192         ['conv2d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_22 (Activation)     (None, 35, 35, 64)   0           ['batch_normalization_22[0][0]'] 
                                                                                                  
 conv2d_20 (Conv2D)             (None, 35, 35, 48)   13824       ['mixed1[0][0]']                 
                                                                                                  
 conv2d_23 (Conv2D)             (None, 35, 35, 96)   55296       ['activation_22[0][0]']          
                                                                                                  
 batch_nor

                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 17, 17, 288)  0          ['mixed2[0][0]']                 
                                                                                                  
 mixed3 (Concatenate)           (None, 17, 17, 768)  0           ['activation_26[0][0]',          
                                                                  'activation_29[0][0]',          
                                                                  'max_pooling2d_2[0][0]']        
                                                                                                  
 conv2d_34 (Conv2D)             (None, 17, 17, 128)  98304       ['mixed3[0][0]']                 
                                                                                                  
 batch_normalization_34 (BatchN  (None, 17, 17, 128)  384        ['conv2d_34[0][0]']              
 ormalizat

                                                                  'activation_39[0][0]']          
                                                                                                  
 conv2d_44 (Conv2D)             (None, 17, 17, 160)  122880      ['mixed4[0][0]']                 
                                                                                                  
 batch_normalization_44 (BatchN  (None, 17, 17, 160)  480        ['conv2d_44[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_44 (Activation)     (None, 17, 17, 160)  0           ['batch_normalization_44[0][0]'] 
                                                                                                  
 conv2d_45 (Conv2D)             (None, 17, 17, 160)  179200      ['activation_44[0][0]']          
          

 ormalization)                                                                                    
                                                                                                  
 activation_54 (Activation)     (None, 17, 17, 160)  0           ['batch_normalization_54[0][0]'] 
                                                                                                  
 conv2d_55 (Conv2D)             (None, 17, 17, 160)  179200      ['activation_54[0][0]']          
                                                                                                  
 batch_normalization_55 (BatchN  (None, 17, 17, 160)  480        ['conv2d_55[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_55 (Activation)     (None, 17, 17, 160)  0           ['batch_normalization_55[0][0]'] 
          

                                                                                                  
 batch_normalization_65 (BatchN  (None, 17, 17, 192)  576        ['conv2d_65[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_65 (Activation)     (None, 17, 17, 192)  0           ['batch_normalization_65[0][0]'] 
                                                                                                  
 conv2d_61 (Conv2D)             (None, 17, 17, 192)  147456      ['mixed6[0][0]']                 
                                                                                                  
 conv2d_66 (Conv2D)             (None, 17, 17, 192)  258048      ['activation_65[0][0]']          
                                                                                                  
 batch_nor

                                                                                                  
 conv2d_70 (Conv2D)             (None, 17, 17, 192)  147456      ['mixed7[0][0]']                 
                                                                                                  
 conv2d_74 (Conv2D)             (None, 17, 17, 192)  258048      ['activation_73[0][0]']          
                                                                                                  
 batch_normalization_70 (BatchN  (None, 17, 17, 192)  576        ['conv2d_70[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_74 (BatchN  (None, 17, 17, 192)  576        ['conv2d_74[0][0]']              
 ormalization)                                                                                    
          

 batch_normalization_76 (BatchN  (None, 8, 8, 320)   960         ['conv2d_76[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_78 (Activation)     (None, 8, 8, 384)    0           ['batch_normalization_78[0][0]'] 
                                                                                                  
 activation_79 (Activation)     (None, 8, 8, 384)    0           ['batch_normalization_79[0][0]'] 
                                                                                                  
 activation_82 (Activation)     (None, 8, 8, 384)    0           ['batch_normalization_82[0][0]'] 
                                                                                                  
 activation_83 (Activation)     (None, 8, 8, 384)    0           ['batch_normalization_83[0][0]'] 
          

                                                                                                  
 activation_91 (Activation)     (None, 8, 8, 384)    0           ['batch_normalization_91[0][0]'] 
                                                                                                  
 activation_92 (Activation)     (None, 8, 8, 384)    0           ['batch_normalization_92[0][0]'] 
                                                                                                  
 batch_normalization_93 (BatchN  (None, 8, 8, 192)   576         ['conv2d_93[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_85 (Activation)     (None, 8, 8, 320)    0           ['batch_normalization_85[0][0]'] 
                                                                                                  
 mixed9_1 

In [11]:
#Dataset augmentation
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
#Fetching trained data path
training_set = train_datagen.flow_from_directory(TRAIN_PATH,
                                                shuffle=True,
                                                 target_size = (299, 299),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')

Found 400 images belonging to 5 classes.


In [13]:
#Fetching validation data path
validation_set = test_datagen.flow_from_directory(VALIDATION_PATH,
                                                 target_size = (299, 299),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')

Found 100 images belonging to 5 classes.


In [14]:
#Model compiling
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy']
)

In [15]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
ckpoint = 'F:/FAHIM/Q4S10/fahim models/check_point.h5'
mc = ModelCheckpoint(ckpoint, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)

In [16]:
#Model training

trained_model = model.fit(
  training_set,
  validation_data = validation_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(validation_set),
  callbacks=[mc]
)


Epoch 1/50
7/7 [==============================] - ETA: 0s - loss: 1.3946 - accuracy: 0.4475
Epoch 1: val_accuracy improved from -inf to 0.73000, saving model to F:/FAHIM/Q4S10/fahim models\check_point.h5
7/7 [==============================] - 76s 7s/step - loss: 1.3946 - accuracy: 0.4475 - val_loss: 0.9168 - val_accuracy: 0.7300
Epoch 2/50
7/7 [==============================] - ETA: 0s - loss: 0.4670 - accuracy: 0.8625
Epoch 2: val_accuracy improved from 0.73000 to 0.82000, saving model to F:/FAHIM/Q4S10/fahim models\check_point.h5
7/7 [==============================] - 37s 5s/step - loss: 0.4670 - accuracy: 0.8625 - val_loss: 0.5594 - val_accuracy: 0.8200
Epoch 3/50
7/7 [==============================] - ETA: 0s - loss: 0.0975 - accuracy: 0.9800
Epoch 3: val_accuracy improved from 0.82000 to 0.88000, saving model to F:/FAHIM/Q4S10/fahim models\check_point.h5
7/7 [==============================] - 35s 6s/step - loss: 0.0975 - accuracy: 0.9800 - val_loss: 0.2776 - val_accuracy: 0.8800
E

KeyboardInterrupt: 

In [ ]:
#Model performance plotting
plt.plot(trained_model.history['loss'], label='train loss')
plt.plot(trained_model.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')
plt.plot(trained_model.history['accuracy'], label='train acc')
plt.plot(trained_model.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

# restoring to the best model
from tensorflow.keras.models import load_model
model = load_model(ckpoint)
loss, acc = model.evaluate(validation_set, steps=3, verbose=0)
print("Validation Accuracy = "+str(acc))

In [ ]:
#Model performance on validation dataset
y_pred = model.predict(validation_set)

In [ ]:
#Calling best match array
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
#Showing best match array
y_pred

In [ ]:
#Importing libraries
from sklearn.metrics import confusion_matrix,accuracy_score

#Confusing matrix plotting
truth_chart = confusion_matrix(validation_set.classes, y_pred)
print(truth_chart)

#Validation accuracy
accuracy = accuracy_score(validation_set.classes, y_pred)
print(accuracy)

In [ ]:
#Validation peformance array
y_pred

In [ ]:
#Model saving
from datetime import datetime
import time

# current timestamp
dt = datetime.now()
x= datetime.timestamp(dt)
# convert to datetime
date_time = datetime.fromtimestamp(x)
str_time = date_time.strftime("%H%M_%d%B%Y")
acc =round(acc*100,3)
dire = 'F:/FAHIM/Q4S10/fahim models/' 
file_name ='Bean_Q4S10_inception_fine_tuned_acc_' +str(acc)+'_'+str_time+'.h5'
loc = dire+file_name
model.save(loc)

In [ ]:
#Model loading
from tensorflow.keras.models import load_model
model=load_model(loc)

In [ ]:
#Test image inputting
TEST_IMAGE = 'F:/FAHIM/Q4S10/fahim datasets/test sets/test online bean pod borer/borer (1).jpeg'
img=image.load_img(TEST_IMAGE, target_size=(299, 299))

In [ ]:
#Test image to array conversion
img_arr=image.img_to_array(img)
img_arr

In [ ]:
#Test image shape
img_arr.shape

In [ ]:
#Test iamge floating point
img_arr=img_arr/255

In [ ]:
#Test image dimension and preprocessing
img_arr=np.expand_dims(img_arr,axis=0)
img_data=preprocess_input(img_arr)
img_data.shape

In [ ]:
#Model prediction on test image
pred = model.predict(img_data)
print(pred)

In [ ]:
#Best prediction counts on test image with probability 
#classes = np.array(['anthracnose', 'mosaic virus', 'unrelated_image'])
import os
classes= np.array(os.listdir(TRAIN_PATH))
print(classes)
top_2 = np.argsort(pred[0])[:-3:-1]
for i in range(2):
    print("{}".format(classes[top_2[i]])+" ({:.5})".format(pred[0][top_2[i]]))
print(pred)
itemindex = np.where(pred == np.max(pred))
print("probability:" + str(np.max(pred)) + "\n" + classes[itemindex[1][0]])

In [ ]:
#Model overall testing

#importing libraries
import cv2#, imutils
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_v3 import preprocess_input
import matplotlib.pyplot as plt
import os
from os import listdir

#load trained model
#loc = "F:/FAHIM/Q4S10/fahim models/Bean_Q4S10_inception_fine_tuned_acc_97.0_1710_12December2022.h5"
model = load_model(loc)

#data classesF:\fahim datasets\Potato\train
#TRAIN_PATH = 'F:/FAHIM/Q4S10/fahim datasets/Potato/train'
classes= np.array(os.listdir(TRAIN_PATH))
print("Diseases : {}",classes)
count = 0
t=0 
# get the path or directory
folder_dir = "F:/FAHIM/Q4S10/fahim datasets/test sets/test online bean aphid/"
test_disease = "aphid"         # ['aphid' 'leafminer' 'mosaic_virus' 'pod borer' 'unrelated_image']

for images in os.listdir(folder_dir):
    # check if the image ends with png or jpg or jpeg
    if (images.endswith(".png") or images.endswith(".jpg") or images.endswith(".jpeg")):
        
        test_image = folder_dir+images
        
        #test image processing
        img=image.load_img(test_image,target_size=(299, 299))     #load images
        img_arr=image.img_to_array(img)                       #image conversion to array
        img_arr=img_arr/255                                   #image-array casting conversion
        img_arr=np.expand_dims(img_arr,axis=0)                #image dimension adjusting

        #model prediction testing
        pred = model.predict(img_arr)
        #data class probability detection
        top_2 = np.argsort(pred[0])[:-3:-1]
        #for i in range(2):
            #print("{}".format(classes[top_2[i]])+" ({:.5})".format(pred[0][top_2[i]]))
        #print(pred)
        itemindex = np.where(pred == np.max(pred))
        #print("probability:" + str(np.max(pred)) + "=>" + classes[itemindex[1][0]])
        t+=1
        if str(classes[itemindex[1][0]]) ==  test_disease:
            count+=1
        else:
            print(images+" probability:" + str(np.max(pred)) + "=>" + classes[itemindex[1][0]])
percent = round(count/t*100,4)
print("out of {} images accuracy {}% ".format(t,percent))